In [3]:
from langchain_groq import ChatGroq


In [4]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key='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',
    model_name="llama-3.1-70b-versatile"
)
response=llm.invoke("The first person to land on earth was?")
print(response.content)

I think there might be some confusion.  The first person to land on Earth would be the first human to be born on the planet. However, if you're asking about the first person to land on Earth from space, that would be Yuri Gagarin, a Soviet cosmonaut who became the first human to journey into outer space on April 12, 1961. He landed safely back on Earth after completing one orbit around the planet.

If you're asking about the first person to walk on Earth, that would be a bit more complicated, as the earliest humans, such as Homo sapiens, evolved around 300,000 years ago in Africa. However, the first person to walk on Earth is not a specific individual that can be identified.

If you could provide more context or clarify your question, I'd be happy to try and help further.


In [5]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.naukri.com/job-listings-software-engineer-epam-systems-hyderabad-pune-bengaluru-3-to-7-years-020924010608?src=seo_srp&sid=17252800837061711&xp=1&px=1")
page_data=loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.










































































 




 










   New User: Apply to selected job without registration 


Cancel



   New User: Apply to selected job and register 
Please fill in the below details to apply to the selected job.
			If you are registred User, Login to Apply


Cancel








Please enter your email ID to get job alerts



 Cancel










Email ID
Username


Password


Invalid ID or Password

 Forgot Password?









 


 




Home
About Us
Jobs
Contact Us
Visit Us


Share On






















Software Engineer
From 3 to 7 year(s) of experienceHyderabad, Pune, Bengaluru  ₹ Not Disclosed by Recruiter



Login to Apply

Apply without Registration
Openings: 3Posted: Just NowJob Applicants: 18Job Views: 136 Send me  Jobs like this
Job Description
MUST HAVE: AT LEAST ONE COLLIBRA IMPLEMENTATION PROJECTJob Description:Seeking a well rounded Collibra SME with 5+ years of experience, to support a broad range of Collibra capabiliti

In [6]:
from langchain_core.prompts import PromptTemplate
prompt_extract=PromptTemplate.from_template(
    """ 
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The Scraped text is from the careers page of a website.
    Your job is to extract the job postings and return them in json format containing 
    the following keys :'role','experience','skills' and 'description'.
    only return the valid json.
    ### VALID JSON(NO PREAMBLE):
    """

)
chain_extract=prompt_extract| llm
res=chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [7]:
from langchain_core.output_parsers import JsonOutputParser

json_parser=JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Software Engineer',
 'experience': '3 to 7 year(s)',
 'skills': ['Integration', 'Collibra', 'Workflow', 'Data Integration Tools'],
 'description': "Seeking a well rounded Collibra SME with 5+ years of experience, to support a broad range of Collibra capabilities: metadata ingestion, data dictionaries, taxonomies, and data lineage to support data governance policies and standards;Configure Collibra workflows,Customize Collibra data models and integrations to align with reporting requirements and industry standardsDevelop and maintain documentation related to Collibra configurations, processes, and workflows, ensuring compliance with internal policies and regulatory mandates Proficient in Agile ways of workingAbility to multi-task and participate in multiple large and complex efforts or projects.Fluent in multiple programming languages and development stacks. Knowledgeable of multiple systems to design more resilient and maintainable solutions.Expert in test automation with dem

In [8]:
type(json_res)


dict

In [9]:
import pandas as pd
df=pd.read_csv("My_Portfolio.csv")
df

,TechStack,Links
0,"MongoDB,ExpressJS,NodeJS, ReactJS",https://github.com/Jansu1/AlumniNetwork
1,"Android , Kotlin, JavaScript",https://github.com/Jansu1/Suraksha-Malware-Det...
2,"StreamLit, Python , Mistral7B, ChromaDB",https://github.com/Jansu1/RAG-Chatbot
3,React -Single Page Application,https://github.com/Jansu1/Time-Zone-Converter
4,"Html,CSS,Bootstrap",https://github.com/Jansu1/ProjectAlumni


In [10]:
import chromadb
import uuid

client=chromadb.PersistentClient('vectorstore')
collection=client.get_or_create_collection(name="portfolio")
if not collection.count():
    for _,row in df.iterrows():
        collection.add(documents=row["TechStack"],
                       metadatas={"links":row["Links"]},
                       ids=[str(uuid.uuid4())])

In [14]:
job=json_res
job['skills']

['Integration', 'Collibra', 'Workflow', 'Data Integration Tools']

In [27]:
links = collection.query(query_texts=job['skills'] ,n_results=2).get('metadatas', [])
links

[[{'links': 'https://github.com/Jansu1/AlumniNetwork'},
  {'links': 'https://github.com/Jansu1/ProjectAlumni'}],
 [{'links': 'https://github.com/Jansu1/RAG-Chatbot'},
  {'links': 'https://github.com/Jansu1/ProjectAlumni'}],
 [{'links': 'https://github.com/Jansu1/RAG-Chatbot'},
  {'links': 'https://github.com/Jansu1/AlumniNetwork'}],
 [{'links': 'https://github.com/Jansu1/RAG-Chatbot'},
  {'links': 'https://github.com/Jansu1/AlumniNetwork'}]]

In [28]:
prompt_email = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION:
    {job_description}
    
    ### INSTRUCTION:
    You are Sai Sujan Shamarthi, a dedicated Computer Science engineering student at Keshav Memorial Institute of Technology, Hyderabad, India. You have a robust understanding of computer science principles, coupled with strong skills in data structures and algorithms. Your project experience spans across AI, cybersecurity, and web development, including:

    - A RAG-based chatbot
    - A Malware Detection system developed using JavaScript for backend and Kotlin for frontend.
    - An Alumni Network application built using MongoDB, Node.js, Express.js and React.js .
    - A Time Zone Converter implemented as a single-page application

    Your task is to draft a cold email to the hiring manager for the specified job role. Highlight your skills and experiences, focusing on your problem-solving abilities and enthusiasm for AI and innovative solutions. Demonstrate how your background aligns with the company’s focus on AI consulting and how you can contribute to their objectives.

    Additionally, include the most relevant projects from the following links to showcase your work: {link_list}
    
    Remember, you are Sai Sujan Shamarthi. 
    Do not provide a preamble.
    ### EMAIL (NO PREAMBLE):
    
    """
)

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)


Subject: Application for Software Engineer Role - Bringing AI Expertise and Innovative Solutions

Dear Hiring Manager,

I am excited to apply for the Software Engineer position at your esteemed organization, as advertised. With a strong foundation in computer science principles and a passion for AI, I am confident that my skills and experience make me an ideal fit for this role.

As a dedicated student at Keshav Memorial Institute of Technology, I have developed a robust understanding of data structures and algorithms, which I believe are essential for driving innovation in software engineering. My project experience spans across AI, cybersecurity, and web development, showcasing my ability to adapt to diverse technologies and domains.

I am particularly drawn to this role because of the emphasis on Collibra and data integration tools. Although I don't have direct experience with Collibra, my experience with MongoDB, Node.js, Express.js, and React.js has given me a solid understanding 